In [1]:
import re
from typing import Dict, List
import uuid
from python.embed_anything import embed_anything
import time
from python.embed_anything import EmbedConfig, BertConfig, EmbedData
from python.embed_anything.vectordb import Adapter
from pinecone import Pinecone, ServerlessSpec
import os


/home/akshay/miniconda3/envs/embed/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
class PineconeAdapter(Adapter):
    """
    Adapter class for interacting with Pinecone, a vector database service.
    """

    def __init__(self, api_key: str):
        """
        Initializes a new instance of the PineconeAdapter class.

        Args:
            api_key (str): The API key for accessing the Pinecone service.
        """
        super().__init__(api_key)
        self.pc = Pinecone(api_key=self.api_key)
        self.index_name = None

    def create_index(
        self,
        dimension: int,
        metric: str = "cosine",
        index_name: str = "anything",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    ):
        """
        Creates a new index in Pinecone.

        Args:
            dimension (int): The dimensionality of the embeddings.
            metric (str, optional): The distance metric to use for similarity search. Defaults to "cosine".
            index_name (str, optional): The name of the index. Defaults to "anything".
            spec (ServerlessSpec, optional): The serverless specification for the index. Defaults to AWS in us-east-1 region.
        """
        self.index_name = index_name
        self.pc.create_index(
            name=index_name, dimension=dimension, metric=metric, spec=spec
        )

    def delete_index(self, index_name: str):
        """
        Deletes an existing index from Pinecone.

        Args:
            index_name (str): The name of the index to delete.
        """
        self.pc.delete_index(name=index_name)

    def convert(self, embeddings: List[EmbedData]) -> List[Dict]:
        """
        Converts a list of embeddings into the required format for upserting into Pinecone.

        Args:
            embeddings (List[EmbedData]): The list of embeddings to convert.

        Returns:
            List[Dict]: The converted data in the required format for upserting into Pinecone.
        """
        data_emb = []

        for embedding in embeddings:
            data_emb.append(
                {
                    "id": str(uuid.uuid4()),
                    "values": embedding.embedding,
                    "metadata": {
                        "text": embedding.text,
                        "file": re.split(
                            r"/|\\", embedding.metadata.get("file_name", "")
                        )[-1],
                    },
                }
            )
        return data_emb

    def upsert(self, data: List[Dict]):
        """
        Upserts data into the specified index in Pinecone.

        Args:
            data (List[Dict]): The data to upsert into Pinecone.

        Raises:
            ValueError: If the index has not been created before upserting data.
        """

        data = self.convert(data)

        if not self.index_name:
            raise ValueError("Index must be created before upserting data")
        self.pc.Index(name=self.index_name).upsert(data)



In [3]:
bert_config = BertConfig(
    model_id="sentence-transformers/all-MiniLM-L12-v2", chunk_size=100
)
embed_config = EmbedConfig(bert=bert_config)


In [4]:

api_key = os.environ.get("PINECONE_API_KEY")
index_name = "anything"
pinecone_adapter = PineconeAdapter(api_key)

try:
    pinecone_adapter.delete_index("anything")
except:
    pass

pinecone_adapter.create_index(dimension=384, metric="cosine")




In [5]:

# data = embed_anything.embed_file(
#     "../test_files/test.pdf",
#     embeder="Bert",
#     config=embed_config,
#     adapter=pinecone_adapter,
# )

data = embed_anything.embed_directory(
    "../test_files", embeder="Bert", extensions=["pdf"], config=embed_config, adapter = pinecone_adapter
)

Unicode mismatch true f_i "fi" Ok("ﬁ") [64257]
Unicode mismatch true f_f_i "ffi" Ok("ﬃ") [64259]
Unicode mismatch true f_f "ff" Ok("ﬀ") [64256]
Unicode mismatch true f_l "fl" Ok("ﬂ") [64258]
Unicode mismatch true f_i "fi" Ok("ﬁ") [64257]
Unicode mismatch true f_f_i "ffi" Ok("ﬃ") [64259]
Unicode mismatch true f_f "ff" Ok("ﬀ") [64256]
Unicode mismatch true f_i "fi" Ok("ﬁ") [64257]
Unicode mismatch true f_f_i "ffi" Ok("ﬃ") [64259]
Unicode mismatch true f_f "ff" Ok("ﬀ") [64256]
Unicode mismatch true f_i "fi" Ok("ﬁ") [64257]
Unicode mismatch true f_f_i "ffi" Ok("ﬃ") [64259]
Unicode mismatch true f_f "ff" Ok("ﬀ") [64256]
Unicode mismatch true f_l "fl" Ok("ﬂ") [64258]
Unicode mismatch true f_i "fi" Ok("ﬁ") [64257]
Unicode mismatch true f_f_i "ffi" Ok("ﬃ") [64259]
Unicode mismatch true f_f "ff" Ok("ﬀ") [64256]
Unicode mismatch true f_i "fi" Ok("ﬁ") [64257]
Unicode mismatch true f_f_i "ffi" Ok("ﬃ") [64259]
Unicode mismatch true f_f "ff" Ok("ﬀ") [64256]
missing char 33 in unicode map {} for <</T

In [6]:
print(data)
print(pinecone_adapter.pc.Index(name="anything").list_paginated())

None
{'namespace': '',
 'pagination': {'next': 'eyJza2lwX3Bhc3QiOiIyNmRiZWRhMS03MDUwLTRiM2EtODJhNy1hNzE3YjQyYzVjZGIiLCJwcmVmaXgiOm51bGx9'},
 'usage': {'read_units': 1},
 'vectors': [{'id': '00549588-1a7a-45ff-bd02-3c9da54c7498'},
             {'id': '00f15247-aef5-4bd7-8487-b6021f3b06cf'},
             {'id': '00f8742a-9eb2-4e96-bf51-ce7c5f06ba3b'},
             {'id': '016a8251-076d-48f0-959d-ee1e953194e4'},
             {'id': '02192d2b-d5dd-482e-a612-baa0426fb4c7'},
             {'id': '02be3e5f-15a6-444d-ba8c-78216bdf2ee0'},
             {'id': '02f0cd37-c1df-4c0d-ba63-71c972ff53e4'},
             {'id': '04889db6-1343-45d1-830b-26e6b4545887'},
             {'id': '04bd8a20-0b86-47e3-821b-3eaf76a43f76'},
             {'id': '050d1471-5702-4a60-b391-6f958ec9a047'},
             {'id': '0527b5ca-8d91-4962-b88d-91010737bfe2'},
             {'id': '052f1484-4069-46b5-a2cd-766867ebeb5a'},
             {'id': '053a8390-5f59-4c42-bacc-39171a6f99fd'},
             {'id': '0583ebcb-2335-414